In [113]:
from math import *
from numpy import *
from scipy import integrate
from scipy.interpolate import InterpolatedUnivariateSpline
fd = open("random", "r")
sd = open("bao", "w")

In [108]:
# Values for the cosmic parameters
c = 3.0*10**5
Om = 0.3
H0 = 72.0
Ol = 0.7
rty = 2.0
# Formula for radial diameter distance
func = lambda z: 1.0/sqrt(Om*(1.0 + z)**3 +0.7)
listyy = []
listss = []
ss = 0.01
# Integrating the radial distance and forming the angular
# diameter distance with a spline of the radial distance
while ss <= rty:
    y, err = integrate.quad(func, 0.0, ss)
    listss.append(ss)
    listyy.append(y)
    ss = ss + 0.01
Hz = InterpolatedUnivariateSpline(listss, listyy)
def angdist(zz):
    value = c*(1.0 + zz)/H0 * Hz(zz)
    return(value)

listxD = []; listyD = []; listzD = []; weightD = []; listxR = []; listyR = []; listzR = []
weightR = []; a=1; b=2; n=0; k=0

In [116]:
%%time
import pandas as pd
df_lrg=pd.read_csv("lrg", sep=' ',header=None)
red = df_lrg.iloc[:,0]
angle = df_lrg.iloc[:,1]
dec = df_lrg.iloc[:,2]
weightD = df_lrg.iloc[:,3]
dist = [angdist(r) for r in red]
dd = [((pi/2.0) - d) for d in dec] # Converting declination into polar angle
# Converting into spherical coordinates
listxD = dist*cos(angle)*sin(dd)
listyD = dist*sin(angle)*sin(dd)
listzD = dist*cos(dd)
n=len(listxD)
df_rnd=pd.read_csv("random", sep=',',header=None)
red = df_rnd.iloc[:,0]
angle = df_rnd.iloc[:,1]
dec = df_rnd.iloc[:,2]
weightD = df_rnd.iloc[:,3]
dist = [angdist(r) for r in red]
dd = [((pi/2.0) - d) for d in dec] # Converting declination into polar angle
# Converting into spherical coordinates
listxR = dist*cos(angle)*sin(dd)
listyR = dist*sin(angle)*sin(dd)
listzR = dist*cos(dd)
n=len(listxR)

0   -4.853108e+10
1    4.951450e+09
2   -3.493681e+10
3    4.717214e+10
4    1.266730e+11
Name: 1, dtype: float64
[  1.31664070e+10   2.52698943e+10   1.53994313e+10   2.38508399e+10
   5.46668937e+10]
[]
CPU times: user 6.41 s, sys: 107 ms, total: 6.52 s
Wall time: 6.52 s


In [117]:
fuzzy = 0
compare = 100000
bins = 201
# This is the size of the bins
size = 1.0
counter = 0
listD = bins*[0]
listR = bins*[0]
listDR = bins*[0]

In [114]:
# To reduce computation time, instead of checking if distances are less than 200 Mpc,
# instead check off against 200^2. 
# This prevents the code from unnecessarily doing a square root for every calculation.comp = 40000.0
m=0
while m < n:
    x0 = listxD[m]
    y0 = listyD[m]
    z0 = listzD[m]
    w = weightD[m]
    # Setting the parameter space around the origin galaxy
    u = x0 + 200.0
    i = x0 - 200.0
    o = y0 + 200.0
    p = y0 - 200.0
    r = z0 + 200.0
    t = z0 - 200.0
    listxt = []
    listyt = []
    listzt = []
    wwt = []
    oo = 0
    for j in range(m + 1, n):
        x1 = listxD[j]
        y1 = listyD[j]
        z1 = listzD[j]
        ww1 = weightD[j]
        # Checking to see which galaxies are within the volume demarcated around
        # the origin galaxy
        if i < x1 < u and p < y1 < o and t < z1 < r:
            listxt.append(x1)
            listyt.append(y1)
            listzt.append(z1)
            wwt.append(ww1)
            oo = oo + 1
    for e in range(0, oo):
        x2 = listxt[e]
        y2 = listyt[e]
        z2 = listzt[e]
        ww2 = wwt[e]
        # Calculating the distance from the x, y, z coordinates
        dd = (x0 - x2)**2 + (y0 - y2)**2 + (z0 - z2)**2
        
        if dd <= comp:
            # Checking to see which bin the distance is to be assigned to
            ds = int(sqrt(dd)/size)
            io = ww2 * w
            listD[ds] = listD[ds] + io
            counter = counter + 1
            fuzzy = fuzzy + 1
            if fuzzy == compare:
                print fuzzy
                compare = compare + 100000
        m=m+1 # --- is this the rught indent?
        
        

[]
[]
[]
